In [1]:
import pandas as pd
driver_id = pd.read_csv("/home/ruijin/Downloads/driver_ids.csv")
ride_ts = pd.read_csv("/home/ruijin/Downloads/ride_timestamps.csv")
ride_id = pd.read_csv("/home/ruijin/Downloads/ride_ids.csv")

In [2]:
ride_ts_dict={}
for counter,elem in enumerate(ride_ts['ride_id']):
    if elem not in ride_ts_dict:
        ride_ts_dict[elem] = ride_ts['ride_picked_up_at'][counter]
    else:
        print('Error')

driver_id_dict={}
for counter,elem in enumerate(driver_id['driver_id']):
    if elem not in driver_id_dict:
        driver_id_dict[elem] = driver_id['driver_onboard_date'][counter]
    else:
        print('Error')

In [3]:
import time
import datetime
driver_ride_id_dict={}
unknown_ride_ide_count, unknown_driver_ide_count=0, 0
for counter,elem in enumerate(ride_id['driver_id']):
    if ride_id['ride_id'][counter] not in ride_ts_dict:
#         print('Unkown ride_id (no mapping between driver_id and ride_id)')
        unknown_ride_ide_count+=1
        continue
        
    if elem not in driver_id_dict:
        unknown_driver_ide_count+=1
        continue
    
    onboard_time = time.mktime(datetime.datetime.strptime(driver_id_dict[elem], "%Y-%m-%d %H:%M:%S").timetuple())
    thisride_time= time.mktime(datetime.datetime.strptime(ride_ts_dict[ride_id['ride_id'][counter]], "%Y-%m-%d %H:%M:%S").timetuple())
    time_range = (thisride_time-onboard_time)/3600.0/24.0
    
    if elem not in driver_ride_id_dict:
        driver_ride_id_dict[elem]=[[ride_id['ride_id'][counter],ride_id['ride_distance'][counter],
                            ride_id['ride_duration'][counter],ride_id['ride_prime_time'][counter],
                            time_range]]
    else:
        driver_ride_id_dict[elem].append([ride_id['ride_id'][counter],ride_id['ride_distance'][counter],
                                  ride_id['ride_duration'][counter],ride_id['ride_prime_time'][counter],
                                  time_range]) 


In [23]:
driver_freq_dict={} #drive freqent and last time
for elem in driver_ride_id_dict.keys():
    driver_freq_dict[elem]=[]
    max_range = 0
    for counter,items in enumerate(driver_ride_id_dict[elem]):
        max_range = max(max_range,items[-1])
        
    driver_freq_dict[elem]=[counter,max_range,float(format(counter/max_range,'.2f'))]
driver_freq_dict        

{'002be0ffdc997bd5c50703158b7c2491': [276, 86.42237268518518, 3.19],
 '007f0389f9c7b03ef97098422f902e62': [30, 85.55581018518518, 0.35],
 '011e5c5dfc5c2c92501b8b24d47509bc': [33, 68.84972222222223, 0.48],
 '0152a2f305e71d26cc964f8d4411add9': [190, 64.43396990740742, 2.95],
 '01674381af7edd264113d4e6ed55ecda': [374, 56.54702546296296, 6.61],
 '0213f8b59219e32142711992ca4ec01f': [239, 30.92385416666667, 7.73],
 '021e5cd15ef0bb3ec20a12af99e142b3': [52, 49.84256944444444, 1.04],
 '0258e250ca195cc6258cbdc75aecd853': [301, 61.85981481481482, 4.87],
 '028b5a4dcd7f4924ebfabcf2e814c014': [362, 51.71491898148148, 7.0],
 '02d6a6b8a6da15fc219a9570f7ebbe78': [61, 51.629745370370365, 1.18],
 '02e440f6c209206375833cef02e0cbae': [55, 48.351724537037036, 1.14],
 '036f3d94e7c65e4e3574822d31c72656': [203, 76.71833333333333, 2.65],
 '039c5afbca8e03e4c18d9c8ea94140ac': [37, 61.9084375, 0.6],
 '039da9c077e17af98ca8530e4d7975f1': [395, 53.067453703703706, 7.44],
 '03f2b5c74cb89f39e58711699e76bf39': [222, 76.

In [33]:
# driver_freq_dict
ratio = 10
sum_churn_freq,sum_unchurn_freq = 0,0
count_churn,count_unchurn = 0,0
driver_churn_dict, driver_unchurn_dict = {}, {}
for elem in driver_freq_dict.keys():
    if driver_freq_dict[elem][-2]<ratio:
        driver_churn_dict[elem]=driver_freq_dict[elem]
        sum_churn_freq = driver_churn_dict[elem][-1]+sum_churn_freq
        count_churn+=1
    else:
        driver_unchurn_dict[elem]=driver_freq_dict[elem]
        sum_unchurn_freq = driver_unchurn_dict[elem][-1]+sum_unchurn_freq
        count_unchurn+=1

print(count_churn, count_unchurn)
print(sum_churn_freq/count_churn, sum_unchurn_freq/count_unchurn)

26 811
7.385384615384617 3.7544389642416776


In [28]:
sum_churn_freq

192.02000000000004